In [56]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import os
import random
import ast

# directory = "/Users/Srikar/Desktop/Velexi/spectra-ml/data/plots"
directory = os.environ['DATA_DIR']
directory = os.path.join(directory, "plots")
os.chdir(directory)

In [57]:
num_samples = len(os.listdir(os.getcwd()))
img = mpimg.imread(os.path.join(directory,os.listdir(os.getcwd())[0]))
spectrum_height = img.shape[0]
spectrum_width = img.shape[1]

In [58]:
def convertimg(img):
    newimg = np.empty([img.shape[0], img.shape[1]])
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            row = img[i][j]
            newimg[i][j] = (row[0] + row[1] + row[2])/3
    return newimg

In [59]:
data = pd.read_csv("/Users/Srikar/Desktop/Velexi/spectra-ml/lab-notebook/smunukutla/data.csv", sep=",")
record_nums = data.iloc[0, :].astype(int).tolist()
spectrum_names = data.iloc[1, :].tolist()
y = data.iloc[2, :].astype(int).tolist()
y = np.reshape(y, (len(y), 1))
num_samples = len(y)

In [67]:
os.listdir().sort()
print(sorted(os.listdir()))

['Actinolite108.png', 'Actinolite114.png', 'Actinolite120.png', 'Actinolite124.png', 'Actinolite130.png', 'Actinolite142.png', 'Actinolite148.png', 'Actinolite154.png', 'Actinolite158.png', 'Actinolite177.png', 'Actinolite189.png', 'Actinolite195.png', 'Actinolite201.png', 'Actinolite205.png', 'Actinolite229.png', 'Actinolite236.png', 'Alunite662.png', 'Alunite670.png', 'Alunite702.png', 'Alunite736.png', 'Alunite821.png', 'Alunite827.png', 'Alunite833.png', 'Alunite837.png', 'Alunite843.png', 'Alunite868.png', 'Alunite874.png', 'Alunite932.png', 'Alunite992.png', 'Chlorite2614.png', 'Chlorite2620.png', 'Chlorite2626.png', 'Chlorite2645.png', 'Chlorite2651.png', 'Chlorite2660.png', 'Chlorite2680.png', 'Chlorite2710.png', 'Chlorite2714.png', 'Chlorite2731.png', 'Chlorite2748.png']


In [69]:
spectra = np.zeros((num_samples, spectrum_height, spectrum_width))
y = []
i = 0
for name in os.listdir():
    print(name)
    if name.find("Actinolite") != -1:
        y.append(int(0))
    elif name.find("Alunite") != -1:
        y.append(int(1))
    else:
        y.append(int(2))
    img = plt.imread(os.path.join(directory,name)) # os.path.join here, look into timeit, pickle file
    spectra[i] = convertimg(img)
    i += 1

Alunite992.png
Actinolite189.png
Actinolite201.png
Actinolite229.png
Actinolite177.png
Chlorite2680.png
Alunite833.png
Alunite827.png
Actinolite148.png
Actinolite158.png
Alunite837.png
Chlorite2651.png
Chlorite2645.png
Chlorite2731.png
Actinolite205.png
Alunite821.png
Alunite932.png
Actinolite114.png
Alunite702.png
Chlorite2620.png
Alunite662.png
Alunite868.png
Chlorite2626.png
Alunite843.png
Alunite670.png
Actinolite108.png
Actinolite120.png
Chlorite2748.png
Chlorite2614.png
Alunite736.png
Alunite874.png
Actinolite130.png
Actinolite124.png
Chlorite2714.png
Chlorite2660.png
Actinolite142.png
Actinolite195.png
Actinolite154.png
Actinolite236.png
Chlorite2710.png


In [32]:
spectra = spectra.reshape(spectra.shape[0], spectra.shape[1]*spectra.shape[2])

In [33]:
spectra.shape

(40, 124416)

In [34]:
y = np.reshape(y, (len(y), 1))
y.shape

(40, 1)

In [42]:
os.chdir("/Users/Srikar/Desktop/Velexi/spectra-ml/lab-notebook/smunukutla")
fi = open("indices.txt", "r")

for i in range(10):
    train_set_indices = ast.literal_eval(fi.readline())
    test_set_indices = ast.literal_eval(fi.readline())
    dev_set_indices = ast.literal_eval(fi.readline())
    
    for j in train_set_indices:
        j = int(j)
    for k in test_set_indices:
        k = int(k)
    for m in dev_set_indices:
        m = int(m)
        
    print(train_set_indices)
    print(test_set_indices)
    print(dev_set_indices)
    
    train_set = spectra[train_set_indices, :]
    train_labels = y[train_set_indices, :]
    dev_set = spectra[dev_set_indices, :]
    dev_labels = y[dev_set_indices, :]
    test_set = spectra[test_set_indices, :]
    test_labels = y[test_set_indices, :]

    train_labels = train_labels.flatten()
    dev_labels = dev_labels.flatten()
    test_labels = test_labels.flatten()

    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(n_estimators=100, bootstrap=True, criterion='entropy')

    from sklearn.model_selection import train_test_split

    y_new = np.copy(y)
    y_new = np.reshape(y_new, (len(y_new), ))
    X_train, X_test, y_train, y_test = train_test_split(spectra, y_new, test_size=0.2, stratify=y_new)
    # clf.fit(train_set, train_labels)
    clf.fit(X_train, y_train)

    from sklearn.metrics import accuracy_score
    # preds = clf.predict(test_set)
    # print("Accuracy:", accuracy_score(test_labels, preds))
    preds = clf.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, preds))

[1, 15, 30, 26, 24, 31, 8, 3, 5, 37, 23, 39, 18, 16, 25, 32, 27, 12, 33, 20, 19, 34, 14, 22]
[35, 29, 4, 13, 36, 17, 7, 6]
[9, 38, 2, 28, 11, 0, 21, 10]
Accuracy: 1.0
[11, 4, 38, 12, 20, 34, 36, 31, 14, 10, 15, 28, 9, 27, 16, 17, 19, 6, 33, 37, 25, 21, 35, 5]
[29, 32, 18, 2, 24, 8, 7, 13]
[23, 26, 22, 39, 3, 1, 30, 0]
Accuracy: 0.75
[33, 23, 11, 14, 4, 29, 36, 38, 6, 21, 15, 37, 39, 7, 10, 31, 22, 1, 17, 28, 27, 32, 35, 16]
[12, 2, 34, 18, 9, 19, 5, 25]
[30, 13, 24, 26, 8, 3, 0, 20]
Accuracy: 0.875
[19, 22, 20, 25, 18, 8, 13, 31, 35, 26, 6, 30, 33, 32, 24, 4, 10, 1, 7, 5, 15, 14, 0, 11]
[12, 38, 36, 23, 29, 9, 28, 27]
[3, 34, 39, 37, 17, 21, 2, 16]
Accuracy: 0.75
[35, 33, 31, 4, 8, 19, 6, 17, 20, 30, 10, 11, 7, 38, 12, 32, 37, 23, 27, 9, 3, 0, 18, 26]
[39, 22, 36, 28, 16, 25, 14, 15]
[34, 13, 2, 5, 29, 1, 24, 21]
Accuracy: 1.0
[13, 16, 39, 9, 37, 29, 38, 12, 8, 15, 5, 36, 10, 19, 33, 32, 18, 21, 27, 30, 11, 22, 2, 26]
[1, 31, 4, 35, 23, 17, 25, 24]
[34, 6, 3, 14, 20, 28, 7, 0]
Accuracy

In [24]:
# train_labels = train_labels.flatten()
# dev_labels = dev_labels.flatten()
# test_labels = test_labels.flatten()

In [25]:
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(bootstrap=True, criterion='entropy')

In [26]:
# from sklearn.model_selection import train_test_split

# y = np.reshape(y, (len(y), ))
# X_train, X_test, y_train, y_test = train_test_split(spectra, y, test_size=0.2, random_state=42)
# # clf.fit(train_set, train_labels)
# clf.fit(X_train, y_train)

In [27]:
# from sklearn.metrics import accuracy_score
# # preds = clf.predict(test_set)
# # print("Accuracy:", accuracy_score(test_labels, preds))
# preds = clf.predict(X_test)
# print("Accuracy:", accuracy_score(y_test, preds))